In [1]:
#Importing dependencies/libraries

from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import time
import numpy as np

In [2]:
#ON
location_on = ["greater-sudbury", "hamilton", "kingston", "london", "niagara-falls",
            "ottawa", "toronto", "windsor", "kitchener", "peterborough", 
            "sault-ste-marie", "thunder-bay"]

#AB
location_ab = ["edmonton", "calgary", "lethbridge", "red-deer"]

#NS
location_ns = ["halifax"]

#QC
location_qc = ["montreal", "sherbrooke", "trois-rivieres", "quebec"]

#BC
location_bc = ["vancouver", "victoria"]

#MB
location_mb = ["winnipeg"]

#Yelp shows 10 restaurants (not including sponsored ones) per page. 
#The web page can be changed by manipulating the number at after the'start=' line of the url.
pages = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230]

In [3]:
#Empty lists to hold the values
names = []
price_range = []
rating = []
total_reviews = []
location = []
province = []

#Webscraping
browser = Browser('chrome')
#This for loop changes the page on the url
for loc in location_ns:  
    #This for loop changes the location of the url
    for page in pages:  
        
        url = f'https://www.yelp.com/search?find_desc=Restaurants&find_loc={loc}%2C+ns&start={page}'
        browser.visit(url)
        html = browser.html
        soup_obj = soup(html, 'html.parser')      
        
        #Scraping the div that contains the target elements
        res_info = []    
        res_info = soup_obj.find_all("div", class_ = "container__09f24__mpR8_ hoverable__09f24__wQ_on border-color--default__09f24__NPAKY")
        time.sleep(5)
        
        #Extracting the target elements
        for info in res_info:
            #Extracting restaurant name
            try:
                name = info.find("a", class_ = "css-19v1rkv").text
                names.append(name)
            except:
                names.append("No data or change the code")
              
            #Extracting price range
            if info.find("span", {"class":"priceRange__09f24__mmOuH css-chan6m"}):
                prc_range = info.find("span", {"class":"priceRange__09f24__mmOuH css-chan6m"}).text
                price_range.append(prc_range)
            elif info.find("span", {"class":"priceRange__09f24__mmOuH css-1s7bx9e"}):
                prc_range = info.find("span", {"class":"priceRange__09f24__mmOuH css-1s7bx9e"}).text
                price_range.append(prc_range)
            else:
                price_range.append("No data or change the code")
            
            #Extracting ratings
            try:
                rati = info.find("span", {"class":"css-gutk1c"}).text
                rating.append(rati)
            except:
                rating.append("No data or change the code")
            
            #Extracting total number of reviews
            try:
                revs = info.find("span", {"class":"css-chan6m"}).text
                total_reviews.append(revs)
            except:
                total_reviews.append("No data or change the code")
            
            #Assigning location and province
            location.append(loc)
            province.append("NS")
               

In [4]:
#To check the data
len(names)

5

In [5]:
#To check the data
len(price_range)

5

In [6]:
#To check the data
price_range

['$$',
 '$',
 'No data or change the code',
 'No data or change the code',
 'No data or change the code']

In [7]:
#Putting the lists together into one dataframe

yelp_ns_df = pd.DataFrame({
    "restaurant_name" : names,
    "price_range" : price_range,
    "rating" : rating,
    "total_reviews" : total_reviews,
    "location" : location,
    "province" : province
})

yelp_ns_df

,restaurant_name,price_range,rating,total_reviews,location,province
0,The Henley House,$$,4.3,(12 reviews),halifax,NS
1,Beanie s Bistro,$,4.8,(5 reviews),halifax,NS
2,Il Porto - Pizzeria And Restaurant,No data or change the code,4.8,(6 reviews),halifax,NS
3,Toots Take Out,No data or change the code,No data or change the code,,halifax,NS
4,Norse Cove Cafe and Supply,No data or change the code,No data or change the code,,halifax,NS


In [8]:
#Cleaning the dataframe; removing duplicates, replacings specific values with NaN, removing rows with NaN values, and cleaning the "total_reviews" column

yelp_ns_df.drop_duplicates()
yelp_ns_df.replace('No data or change the code', np.nan, inplace = True)
yelp_ns_df.dropna(inplace = True)
yelp_ns_df = yelp_ns_df.reset_index(drop = True)
yelp_ns_df['total_reviews'] = yelp_ns_df['total_reviews'].str.replace(r'[\(\) reviews]', '', regex=True)

yelp_ns_df

,restaurant_name,price_range,rating,total_reviews,location,province
0,The Henley House,$$,4.3,12,halifax,NS
1,Beanie s Bistro,$,4.8,5,halifax,NS


In [9]:
#Creating a csv from the dataframe

yelp_ns_df.to_csv("yelp_ns.csv", index = False)

In [10]:
browser.quit()